In [1]:
!python --version

Python 3.10.0


In [ ]:
import os
import shutil
import tempfile
import monai
import matplotlib.pyplot as plt
from tqdm import tqdm

from monai.losses import DiceCELoss 
from monai.inferers import sliding_window_inference  #input image into model in patch manner
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    ResizeWithPadOrCropd,
    Lambdad,
    SpatialCropd
)

from monai.config import print_config
from monai.metrics import DiceMetric  #to calculate similarity b/w model output and ground truth output
from monai.networks.nets import UNETR, SwinUNETR
from monai.transforms.spatial.functional import resize

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)


import torch

print_config()

In [3]:
torch.cuda.empty_cache()

In [ ]:
def modify_labels(label):
    # Set label 1 to foreground and all other labels to background
    return torch.where(label == 3, torch.tensor(1), torch.tensor(0))

In [5]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-175,
            a_max=250,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        ResizeWithPadOrCropd(
            keys=["image", "label"], 
            spatial_size=(300,250,250),
        ),
        Lambdad(keys=["label"],func=modify_labels),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[0],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[1],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[2],
            prob=0.10,
        ),
        RandRotate90d(
            keys=["image", "label"],
            prob=0.10,
            max_k=3,
        ),
        RandShiftIntensityd(
            keys=["image"],
            offsets=0.10,
            prob=0.50,
        ),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        ResizeWithPadOrCropd(
            keys=["image", "label"], 
            spatial_size=(300,250,250),
        ),
        Lambdad(keys=["label"],func=modify_labels),
    ]
)

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [ ]:

split_json = "dataset.json"
datasets = split_json
# datasets = "/kaggle/input/json-file/dataset_0.json"
datalist = load_decathlon_datalist(datasets, True, "training")
val_files = load_decathlon_datalist(datasets, True, "validation")
train_ds = CacheDataset(
    data=datalist,
    transform=train_transforms,
    cache_num=6,
    cache_rate=1.0,
    num_workers=4,
)
val_ds = CacheDataset(
    data=val_files, 
    transform=val_transforms, 
    cache_num=3, 
    cache_rate=1.0, 
    num_workers=4,
)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)

Loading dataset: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.94s/it]


In [7]:
for image in enumerate(train_loader):
    print("Image:",image[1]["image"].shape)
    print("label:", image[1]["label"].shape)
    break

for image in enumerate(val_loader):
    print("Image:", image[1]["image"].shape)
    print("label:",image[1]["label"].shape)
    break

Image: torch.Size([4, 1, 96, 96, 96])
label: torch.Size([4, 1, 96, 96, 96])
Image: torch.Size([1, 1, 300, 250, 250])
label: torch.Size([1, 1, 300, 250, 250])


In [3]:
# slice_map = {
#     "img0035.nii.gz": 170,
#     "img0036.nii.gz": 70,
#     "img0037.nii.gz": 204,
#     "img0038.nii.gz": 204,
#     "img0039.nii.gz": 204,
#     "img0040.nii.gz": 180,
# }
# case_num = 1
# img_name = os.path.split(val_ds[case_num]["image"].meta["filename_or_obj"])[1]
# img = val_ds[case_num]["image"]
# label = val_ds[case_num]["label"]
# img_shape = img.shape
# label_shape = label.shape
# print(f"image shape: {img_shape}, label shape: {label_shape}")
# plt.figure("image", (18, 6))
# plt.subplot(1, 2, 1)
# plt.title("image")
# plt.imshow(img[0, :, :, slice_map[img_name]].detach().cpu(), cmap="gray")
# plt.subplot(1, 2, 2)
# plt.title("label")
# plt.imshow(label[0, :, :, slice_map[img_name]].detach().cpu())
# plt.show()

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device="cpu"
import numpy as np
import nibabel as nib

model = SwinUNETR(
    in_channels=1,
    out_channels=2,
    img_size=(96, 96, 96),
    feature_size=48,
    spatial_dims= 3,
    dropout_path_rate=0.0,
    drop_rate=0.0,
    norm_name="instance",
    use_checkpoint=True,
    attn_drop_rate=0.0,
).to(device)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
torch.backends.cudnn.benchmark = True
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
def validation(epoch_iterator_val):
    model.eval()
    with torch.no_grad():
        # j=0
        for batch in epoch_iterator_val:
            val_inputs, val_labels = (batch["image"].cuda(), batch["label"].cuda())
            val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 4, model)
            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]

            # for i, tensor in enumerate(val_labels_convert[0]): 
            #     image_array = np.array(tensor.cpu())
            #     nifti_image = nib.Nifti1Image(image_array, affine=None)
            #     output_file_path = os.path.join(f'pancreaticModelInputOutput', f'Val_label00{j}_label{i}.nii.gz')
            #     nib.save(nifti_image, output_file_path)
        
                
            # print("Validation Input Image Saved")
        
            # for i, tensor in enumerate(val_output_convert[0]): 
            #     image_array = np.array(tensor.cpu())
            #     nifti_image = nib.Nifti1Image(image_array, affine=None)
            #     output_file_path = os.path.join(f'pancreaticModelInputOutput', f'Val_labeloutput00{j}_label{i}.nii.gz')
            #     nib.save(nifti_image, output_file_path)
                
            # print("Validation Image Saved")

            # j+=1
            
            dice_metric(y_pred=val_output_convert, y=val_labels_convert)
            epoch_iterator_val.set_description("Validate (%d / %d Steps)" % (global_step, 10.0))  # noqa: B038
        mean_dice_val = dice_metric.aggregate().item()
        dice_metric.reset()
    return mean_dice_val


def train(global_step, train_loader, dice_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    global_step += 1
    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["image"].cuda(), batch["label"].cuda())

        val_labels_list = decollate_batch(y)
        val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description(  # noqa: B038
            "Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss)
        )
        
    if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
            epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True)
            dice_val = validation(epoch_iterator_val)
            print("Dice Value:", dice_val)
            epoch_loss /= step
            epoch_loss_values.append(epoch_loss)
            metric_values.append(dice_val)
            if dice_val > dice_val_best:
                dice_val_best = dice_val
                global_step_best = global_step
                torch.save({
                        'epoch': global_step,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss_function,
                        'metric_values': metric_values,
                        'loss_values': epoch_loss_values}, "2Pretrained_SwinUnetr_Pancreatic.pth")
                print(
                    "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val)
                )
            else:
                print(
                    "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(
                        dice_val_best, dice_val
                    )
                )
    return global_step, dice_val_best, global_step_best

post_label = AsDiscrete(to_onehot=2)
post_pred = AsDiscrete(argmax=True, to_onehot=2)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)


In [ ]:
epoch_loss_values = []
metric_values = []
dice_val_best = 0.0
global_step_best = 0
global_step = 3
max_iterations = 2500
eval_num = 3 

In [1]:
# PATH = "best_model.pth"

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# criterion = checkpoint['loss']
# epoch_loss_values = checkpoint['loss_values']
# metric_values = checkpoint['metric_values']
# global_step = checkpoint['epoch']

# model.eval()

In [ ]:
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(global_step, train_loader, dice_val_best, global_step_best)
    break

In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Iteration Average Loss")
x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [eval_num * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.show()